<h1>Reddit Scraped Comments</h1>

<h3>Installation and import of libraries</h3>

In [1]:
!pip install pyLDAvis

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

import nltk
nltk.download('stopwords') #download if don't have yet
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lindy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h3>Viewing Data</h3>

In [3]:
# Path
folder_path = "../Reddit_Scraped_Comments/"
file1 = "anti_lockdown_comments.csv"
file2 = "corona_lockdown_comments.csv"
file3 = "covid19_lockdown_comments.csv"

In [4]:
anti_lockdown_comments_df = pd.read_csv(folder_path + file1)
anti_lockdown_comments_df

,Unnamed: 0,id,author,title,score,comms_num,created,timestamp,commenter,comment,top_lvl
0,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,schu4KSU,People like this everywhere in the world. Dif...,1
1,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,StupidizeMe,"Well, there's certainly enough of them to caus...",0
2,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,Eltharion-the-Grim,They are largely absent from Asia. The only pl...,0
3,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,Thisam,"Yup, the percentage of population who are easi...",0
4,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,NaN,Short answer: Yes. Look at the USA and how Bre...,0
...,...,...,...,...,...,...,...,...,...,...,...
1516,0,gl3pgs,WearyThanks,Coronavirus: Fears Russian trolls are behind a...,217,48,1.589668e+09,2020-05-17 06:27:03,BigDaddyCoolDeisel,Is there any fucking question anymore?,1
1517,0,gl3pgs,WearyThanks,Coronavirus: Fears Russian trolls are behind a...,217,48,1.589668e+09,2020-05-17 06:27:03,NaN,[removed],1
1518,0,gl3pgs,WearyThanks,Coronavirus: Fears Russian trolls are behind a...,217,48,1.589668e+09,2020-05-17 06:27:03,AutoModerator,Your comment has been removed because\n\n* **P...,0
1519,0,gl3pgs,WearyThanks,Coronavirus: Fears Russian trolls are behind a...,217,48,1.589668e+09,2020-05-17 06:27:03,Tysk_Kille,Fake news,1


In [5]:
corona_lockdown_comments_df = pd.read_csv(folder_path + file2)
corona_lockdown_comments_df

,Unnamed: 0,id,author,title,score,comms_num,created,timestamp,commenter,comment,top_lvl
0,0,hb3v90,hunter15991,Arizona Sheriff Who Refused to Enforce Lockdow...,63153,1166,1.592440e+09,2020-06-18 08:29:20,NaN,[deleted],1
1,0,hb3v90,hunter15991,Arizona Sheriff Who Refused to Enforce Lockdow...,63153,1166,1.592440e+09,2020-06-18 08:29:20,NaN,[removed],0
2,0,hb3v90,hunter15991,Arizona Sheriff Who Refused to Enforce Lockdow...,63153,1166,1.592440e+09,2020-06-18 08:29:20,lyrkyr12345,Joe Arpaio is human garbage,0
3,0,hb3v90,hunter15991,Arizona Sheriff Who Refused to Enforce Lockdow...,63153,1166,1.592440e+09,2020-06-18 08:29:20,Latexfrog,Paul Penzone. I had the great fortune of going...,0
4,0,hb3v90,hunter15991,Arizona Sheriff Who Refused to Enforce Lockdow...,63153,1166,1.592440e+09,2020-06-18 08:29:20,Basslineblues,"Unfortunately, Joe Arpaio is running for sheri...",0
...,...,...,...,...,...,...,...,...,...,...,...
9759,0,jjlxfb,Aryamatha,France expected to impose four-week national l...,4913,802,1.603884e+09,2020-10-28 19:23:22,plentyoffishes,"Won't work, just like the first one didn't work.",1
9760,0,jjlxfb,Aryamatha,France expected to impose four-week national l...,4913,802,1.603884e+09,2020-10-28 19:23:22,BadTiger85,I would expect the same kind of unrest in Fran...,1
9761,0,jjlxfb,Aryamatha,France expected to impose four-week national l...,4913,802,1.603884e+09,2020-10-28 19:23:22,GEM592,Riots in 3 ... 2 ...,1
9762,0,jjlxfb,Aryamatha,France expected to impose four-week national l...,4913,802,1.603884e+09,2020-10-28 19:23:22,VagnerLove,Where was all the bloody prep that we were mea...,1


In [6]:
covid19_lockdown_comments_df = pd.read_csv(folder_path + file3)
covid19_lockdown_comments_df

,Unnamed: 0,id,author,title,score,comms_num,created,timestamp,commenter,comment,top_lvl
0,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,Skooter_McGaven,I wish we had an understanding of where the ma...,1
1,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,Richandler,> Is it overly family spread?\n\nI believe the...,0
2,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,NaN,This sounds horrible and I hate myself a littl...,0
3,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,lcbk,My husband and I are not yet confirmed to have...,0
4,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,ZombiGrn,In my neighborhood people started throwing par...,0
...,...,...,...,...,...,...,...,...,...,...,...
885,0,g8dyy2,StudentjeNL,When to release the lockdown: A wellbeing fram...,231,314,1.587906e+09,2020-04-26 21:02:11,McGloin_the_GOAT,This is an economic analysis and in economics ...,0
886,0,g8dyy2,StudentjeNL,When to release the lockdown: A wellbeing fram...,231,314,1.587906e+09,2020-04-26 21:02:11,Zamaamiro,This sub seems to be attracting more and more ...,1
887,0,g8dyy2,StudentjeNL,When to release the lockdown: A wellbeing fram...,231,314,1.587906e+09,2020-04-26 21:02:11,NaN,"I wouldn't say that's true here. This sub, lik...",0
888,0,g8dyy2,StudentjeNL,When to release the lockdown: A wellbeing fram...,231,314,1.587906e+09,2020-04-26 21:02:11,jphamlore,Fortunately there is another model from IHME t...,1


<h3>Cleaning Data</h3>

In [7]:
# Drop columns with removed comments
anti_lockdown_comments_df = anti_lockdown_comments_df[anti_lockdown_comments_df.comment != "[removed]"]
corona_lockdown_comments_df = corona_lockdown_comments_df[corona_lockdown_comments_df.comment != "[removed]"]
covid19_lockdown_comments_df = covid19_lockdown_comments_df[covid19_lockdown_comments_df.comment != "[removed]"]

In [8]:
list1 = anti_lockdown_comments_df["comment"].tolist()
list2 = corona_lockdown_comments_df["comment"].tolist()
list3 = covid19_lockdown_comments_df["comment"].tolist()
comments = list1 + list2 + list3

# Length of texts / documents
print(len(comments))

# Sample texts
print(comments[0])
print(comments[1])

11581
People like this everywhere in the world.  Difference is, are there enough to gain political power?
Well, there's certainly enough of them to cause a massive surge in COVID-19 cases and deaths, and to take some of us with them.


In [9]:
def remove_special_chars(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, ' ', text)

def remove_digit_strings(text):
    return re.sub(r'\d+', '', text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)

def clean_text(text):
    result_text = remove_links(text)
    result_text = remove_digit_strings(result_text)
    result_text = remove_special_chars(result_text)
    result_text = result_text.lower()
    return result_text

In [10]:
for i in range(len(comments)):
    comments[i] = clean_text(comments[i])

In [11]:
print(comments[0])
print(comments[1])

people like this everywhere in the world   difference is  are there enough to gain political power 
well  there s certainly enough of them to cause a massive surge in covid  cases and deaths  and to take some of us with them 


<h3>Tokenizing Data</h3>

In [12]:
# Stopwords
stop_words = stopwords.words('english')
exclude_words = stop_words

#exclude common words 
exclude_words_extra = ["covid","lockdown","corona","pandemic","let","im","ive","would","one","also","to","say","day","well","month","thing","take","see","get","go"]

exclude_words.extend(exclude_words_extra)

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(comments))

In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['people', 'like', 'this', 'everywhere', 'in', 'the', 'world', 'difference', 'is', 'are', 'there', 'enough', 'to', 'gain', 'political', 'power']


In [15]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in exclude_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['people', 'everywhere', 'world', 'difference', 'enough', 'gain', 'political', 'power']]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


[[('difference', 1),
  ('enough', 1),
  ('everywhere', 1),
  ('gain', 1),
  ('people', 1),
  ('political', 1),
  ('power', 1),
  ('world', 1)]]

<h3>LDA Model</h3>

In [18]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

# supporting function 2
def compute_perplexity_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    perplexity_score = lda_model.log_perplexity(corpus_sets[i])
    
    return perplexity_score

In [19]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"comment" + 0.024*"question" + 0.022*"lock" + 0.022*"post" + '
  '0.021*"enough" + 0.020*"base" + 0.020*"risk" + 0.018*"allow" + '
  '0.016*"source" + 0.016*"cost"'),
 (1,
  '0.029*"death" + 0.018*"population" + 0.016*"year" + 0.014*"come" + '
  '0.011*"long" + 0.011*"hospital" + 0.010*"end" + 0.010*"see" + 0.009*"model" '
  '+ 0.009*"economy"'),
 (2,
  '0.023*"virus" + 0.023*"case" + 0.020*"country" + 0.016*"vaccine" + '
  '0.015*"lockdown" + 0.014*"spread" + 0.011*"variant" + 0.011*"measure" + '
  '0.011*"month" + 0.011*"point"'),
 (3,
  '0.036*"people" + 0.013*"time" + 0.013*"think" + 0.012*"go" + 0.012*"make" + '
  '0.011*"work" + 0.011*"need" + 0.011*"even" + 0.010*"still" + 0.009*"know"')]


In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.8784715829884995

Coherence Score:  0.6165667120922931


In [22]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Lindy\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.367434 -0.009620       1        1  49.425116
2     -0.137665 -0.313928       2        1  23.145829
1     -0.138690  0.214520       3        1  19.028149
0     -0.091079  0.109027       4        1   8.400906, topic_info=           Term         Freq        Total Category  logprob  loglift
4        people  2667.000000  2667.000000  Default  30.0000  30.0000
11        death   811.000000   811.000000  Default  29.0000  29.0000
222       virus   791.000000   791.000000  Default  28.0000  28.0000
8          case   781.000000   781.000000  Default  27.0000  27.0000
22      country   703.000000   703.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
532  discussion    84.414181    84.878039   Topic4  -4.9922   2.4714
789    outbreak    96.400043    96.953645   Topic4  -4.8594   2.4711
314    evidence   131.931757   133.160393   Topic4  -4.5456   2.4676
958    specific   106.793011   107.547790   Topic4  -4.7570   2.4698
20      believe   115.933605   274.718867   Topic4  -4.6749   1.6141

[168 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
959       4  0.999822   action
607       4  0.995879    allow
1829      3  0.994614     area
374       4  0.996735  article
1166      3  0.993811    avoid
...     ...       ...      ...
900       1  0.997887     well
258       4  0.997980    whole
127       1  1.000030     work
7         3  0.998907    world
255       3  0.998338     year

[154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [23]:
optimal_model = lda_model
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.027*"comment" + 0.024*"question" + 0.022*"lock" + 0.022*"post" + '
  '0.021*"enough" + 0.020*"base" + 0.020*"risk" + 0.018*"allow" + '
  '0.016*"source" + 0.016*"cost"'),
 (1,
  '0.029*"death" + 0.018*"population" + 0.016*"year" + 0.014*"come" + '
  '0.011*"long" + 0.011*"hospital" + 0.010*"end" + 0.010*"see" + 0.009*"model" '
  '+ 0.009*"economy"'),
 (2,
  '0.023*"virus" + 0.023*"case" + 0.020*"country" + 0.016*"vaccine" + '
  '0.015*"lockdown" + 0.014*"spread" + 0.011*"variant" + 0.011*"measure" + '
  '0.011*"month" + 0.011*"point"'),
 (3,
  '0.036*"people" + 0.013*"time" + 0.013*"think" + 0.012*"go" + 0.012*"make" + '
  '0.011*"work" + 0.011*"need" + 0.011*"even" + 0.010*"still" + 0.009*"know"')]


In [24]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [25]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(4):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(lda_model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

comment              0.027
question             0.024
lock                 0.022
post                 0.022
enough               0.021
base                 0.020
risk                 0.020
allow                0.018
source               0.016
cost                 0.016
Topic 1 |---------------------

death                0.029
population           0.018
year                 0.016
come                 0.014
long                 0.011
hospital             0.011
end                  0.010
see                  0.010
model                0.009
economy              0.009
Topic 2 |---------------------

virus                0.023
case                 0.023
country              0.020
vaccine              0.016
lockdown             0.015
spread               0.014
variant              0.011
measure              0.011
month                0.011
point                0.011
Topic 3 |---------------------

people               0.036
tim